In [1]:
# -----********************-----

# Created Time: 2024/12/09

# Last updated: 2024/12/11

# Author: Tara Liu, Yiyi He

### Use Case

# This notebook explores the application of autoregressive models
# 1. 

# -----********************-----

# Libraries

In [80]:
# Import libraries
import os
import warnings
warnings.filterwarnings("ignore")

# Stats
from statsmodels.tsa.api import ARDL
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.ardl import ardl_select_order
from statsmodels.tsa.api import VAR

# Geo
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Processing
from tqdm import tqdm

In [17]:
# Define functions
def find_lag(df, maxlag):

    time_start = pd.Timestamp.now()
    
    sel_res = ardl_select_order(
        df['pct_blackout'],
        exog=df[['t2m', 'wind_speed', 'tp']],
        maxlag=maxlag,
        ic='aic',
        maxorder=maxlag,
        causal=True,
        trend='ct'
        )
    
    time_selected = pd.Timestamp.now()
    print(f'time elapsed for selecting order: {time_selected-time_start}')
    
    return sel_res.model.ardl_order

In [5]:
# Load input dataframe
df_518 = pd.read_csv("/Users/yiyi/Desktop/df_for_model.csv")
df_518.head(3)

,datetime,pct_blackout,wind_forest_cover,t2m,tp,wind_speed,station_id,climate_zone_code
0,2014-11-17 17:00:00,0.0,0.0,295.09723,5.645638e-04,1.310930,495,Aw
1,2014-11-29 17:00:00,0.0,0.0,293.04870,0.000000e+00,2.215274,495,Aw
2,2014-11-30 22:00:00,0.0,0.0,289.83447,4.351137e-07,2.000125,495,Aw


In [48]:
s_id = 100
station_df = df_518[df_518['station_id'] == s_id]
max_lag = 5
sel_res = ardl_select_order(
        station_df['pct_blackout'],
        exog=station_df[['t2m', 'wind_speed', 'tp']],
        maxlag=maxlag,
        ic='aic',
        maxorder=maxlag,
        causal=True,
        trend='ct'
        )
sel_res.model.ardl_order

(3, 1, 1)

# FIND LAG

In [28]:
# Load input dataframe
df_518 = pd.read_csv("/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/01_data/2024_9_10_Tingyu/processed/station_518_forest_perc_50kmBu_district_climate_dummy_df.csv",
                     index_col=0)
df_518.head(3)

,datetime,pct_blackout,wind_forest_cover,t2m,tp,wind_speed,station_id,climate_zone_code,forest_perc_50kmbuffer,forest_perc_district,wind_forest_cover_50kmbu,wind_forest_cover_district,Am,Aw,BSh,Cw
0,2014-11-17 17:00:00,0.0,0.0,295.09723,5.645638e-04,1.310930,495,Aw,0.0006,0.0006,0.000787,0.000787,0,1,0,0
1,2014-11-29 17:00:00,0.0,0.0,293.04870,0.000000e+00,2.215274,495,Aw,0.0006,0.0006,0.001329,0.001329,0,1,0,0
2,2014-11-30 22:00:00,0.0,0.0,289.83447,4.351137e-07,2.000125,495,Aw,0.0006,0.0006,0.001200,0.001200,0,1,0,0


In [120]:
# Create a list of unique station ids
station_id_lst = list(set(df_518.station_id.unique()))
# Initiate an empty dictionary
station_id_lag_dic = {}
# Set max lag
maxlag = 12
# Iterate through all stations
for s_id in tqdm(station_id_lst):
    # Subset station data
    station_df = df_518[df_518['station_id'] == s_id].sort_values(by='datetime')
    # Find optimum lag and store station id with optimum lag in dictionary
    sel_res = ardl_select_order(
        station_df['pct_blackout'],
        exog=station_df[['t2m', 'wind_speed', 'tp']],
        maxlag=maxlag,
        ic='bic',
        maxorder=maxlag,
        causal=True,
        trend='ct'
        )
    # optimum lag values for endogenous variables
    endo_res_lst = list(sel_res.aic.values[0][1].values())
    # insert the optimum lag value for exdogenous variable
    endo_res_lst.insert(0, sel_res.aic.values[0][0])
    # store station id and optimum lag values in dictionary
    station_id_lag_dic[s_id] = endo_res_lst # key: station id, value:[pct_blackout_lag, t2m_lag, wind_speed_lag, tp_lag]

  0%|          | 0/516 [00:00<?, ?it/s]

In [103]:
station_id_lag_dic_copy = station_id_lag_dic

In [106]:
station_id_lag_df = pd.DataFrame.from_dict(station_id_lag_dic_copy, orient='index')

In [118]:
station_id_optimum_lag_aic_df = station_id_lag_df.reset_index().rename(columns={
                                                          'index': 'station_id',
                                                          0: 'pct_blackout_lag',
                                                          1: 't2m_lag',
                                                          2: 'wind_speed_lag',
                                                          3: 'tp_lag'
                                                          })
station_id_optimum_lag_aic_df.to_csv('/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/01_data/2024_9_10_Tingyu/processed/station_516_ARDL_optimum_lag_aic_df.csv')

In [119]:
station_id_optimum_lag_aic_df

,station_id,pct_blackout_lag,t2m_lag,wind_speed_lag,tp_lag
0,1,8,1.0,NaN,3.0
1,2,10,10.0,2.0,6.0
2,3,9,10.0,NaN,NaN
3,4,10,6.0,NaN,1.0
4,5,3,12.0,11.0,10.0
5,6,12,12.0,12.0,11.0
6,8,11,11.0,1.0,12.0
7,9,11,1.0,NaN,7.0
8,10,9,9.0,3.0,NaN
9,11,9,11.0,11.0,12.0


# VAR

In [26]:
def testMaxLagAndTime_VAR(df, maxlag):
    '''
    This function is used to test max lag selection for VAR model and record time.
    
    Args:
    df: pd.DataFrame, the dataframe with a cerntain station_id
    maxlag: int, the maximum lag to be tested
    
    Params:
    See var_select_order's parameters
    
    Returns:
    test_model: VAR model, the model with the best maxlag
    '''

    print('Begin testing...')
    print(f'testing maxlag: {maxlag}')
    time_start = pd.Timestamp.now()

    # select order with VAR
    order_selected = VAR(df[['pct_blackout', 't2m', 'wind_speed', 'tp']]).select_order(
        maxlags=maxlag)
    
    # fit VAR model
    time_selected = pd.Timestamp.now()
    print(f'time elapsed for selecting order: {time_selected-time_start}')
    
    test_model = VAR(df[['pct_blackout', 't2m', 'wind_speed', 'tp']]).fit(
        maxlags=maxlag, method='ols', ic='aic', verbose=True, trend='ctt')
    
    # print results
    print(f'order selected: {order_selected}')
    # print aic
    print(f'AIC: {test_model.aic}')
    
    
    time_fit = pd.Timestamp.now()
    print(f'time elapsed for fitting model: {time_fit-time_selected}')

    time_end = pd.Timestamp.now()
    print(f'time elapsed for the whole process: {time_end-time_start}')
    print('End testing...')
    print('model summary:')
    print(test_model.summary())

    return test_model

In [4]:
# Read input csv
df_518 = pd.read_csv('/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/01_data/2024_9_10_Tingyu/processed/station_518_forest_perc_50kmBu_district_climate_dummy_df.csv',
                     index_col=0)

In [5]:
s_id = 100
station_df = df_518[df_518['station_id'] == s_id]
max_lag = 5

In [ ]:
df.head(10)

In [27]:
testMaxLagAndTime_VAR(station_df, 4)

Begin testing...
testing maxlag: 4
time elapsed for selecting order: 0 days 00:00:00.012770
<statsmodels.tsa.vector_ar.var_model.LagOrderResults object. Selected orders are: AIC -> 1, BIC -> 0, FPE -> 1, HQIC ->  0>
Using 1 based on aic criterion
order selected: <statsmodels.tsa.vector_ar.var_model.LagOrderResults object. Selected orders are: AIC -> 1, BIC -> 0, FPE -> 1, HQIC ->  0>
AIC: -15.533333109884271
time elapsed for fitting model: 0 days 00:00:00.010178
time elapsed for the whole process: 0 days 00:00:00.022983
End testing...
model summary:
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 16, Dec, 2024
Time:                     19:46:21
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                   -15.3590
Nobs:                     740.000    HQIC:                  -15.4661
Log likelihood:           1575.28    FPE:                1.794

/Applications/anaconda3/envs/india0/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Applications/anaconda3/envs/india0/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
